# 강화학습
* 가장 흥미진진하며 가장 오래된 분야 1950년 시작
* 2013 딥마인드의 아타리 실현. 기계는 픽셀 데이터만 입력으로 받아 어떤 규칙 사전정보도 없이 사람을 능가함.
* 강화학습에 딥러닝을 적용한 성과.
* 심층 강화학습에서 가장 중요한 두가지 정책 그레디언트, 심층 Q-네트워크를 마르코프 결정과정과 함께 소개한다.
---
## 보상을 최적화하기 위한 학습
* 에이전트는 관측을 하고 환경에서 행동을 한다. 그 결과로 보상을 받는다.
* 양의 보상은 기쁨,음의 보상은 아픔으로 생각. 기쁨을 최대로하는 훈련,
* 보상이 없을 수도 있다. 미로를 움직인다면 스탭마다 음의 보상을 얻기에 가능한한 빨리 탈출구를 찾는 편이 좋다.
---
## 정책 탐색
* 에이전트가 행동을 결정하기 위해 사용하는 알고리즘을 정책이라고 한다.
* 예를들어 관측을 입력으로 받고 수행할 행동을 출력하는 신경망이 정책이 될 수 있다.
* 정책은 생각할 수 있는 어떤 알고리즘도 될 수 있다.
* 무작위성이 있으면 확률적 정책
* 먼지청소기의 경우 움직이는 확률 p와 각도의 범위 r이 있고 이를 정책 파라미터라 한다.
* 모든걸 시도해보고 좋은 성능 조합을 고르는 알고리즘을 생각해본다 하면. 이것은  정책공간이 너무 크면 매우 어렵다.
* 정책탐색의 예로 유전 알고리즘이 있다.
  * 예를들어 1세대 정책 100개를 랜덤하게 생성한 후에 성능 낮은 80개는 버리고 20개를 살려 각각 자식 정책 4개를 생산하게 한다.
  * 이 자식정책은 부모를 복사한 것에 약간의 무작위성을 더한 것이다.
  * 살아남은 정책과 그 자식은 2세대를 구성한다.
* 정책 그레디언트 알고리즘
  * 정책파라미터의 보상 그레디언트를 평가해서 높은 보상의 방향을 따르는 그레디언트 최적화 기법 사용.
---
## OpenAI 짐
* 강화학습의 어려운점은 에이전트 훈련을 위해 작업 환경을 마련해야 한다는 것이다.
* 아타리 훈련을 위한 시뮬레이터가 있어야할것. 보행로봇을 훈련한다면 컴퓨터 시뮬레이션 환경이 필요하다.
  * PyBullet이나 MuJuCo같은 시뮬레이션 라이브러리 사용 가능.
* OpenAI 짐은 다양한 종류의 시뮬레이션 환경을 제공한다.
* 툴킷을 설치하기 전에 환경을 활성화 해야한다.

In [1]:
!apt install xvfd
!python3 -m pip install -U pyvirtualdisplay

Reading package lists... Done
Building dependency tree       
Reading state information... Done
E: Unable to locate package xvfd


In [2]:
import sklearn
import tensorflow as tf
from tensorflow import keras
import numpy as np
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.animation as animation

In [3]:
import gym
env=gym.make("CartPole-v1")
obs=env.reset()
obs

array([0.00893967, 0.03785311, 0.03280713, 0.02609832])

* CartPole환경을 만들었다. 카트 위에 놓인 막대가 넘어지지 안혿록 왼족이나 오른쪽으로 가속할수 있는 2D시뮬레이션이다.
* gym.envs.registry.all()함수를 호출하면 환경의 전체 목록을 얻을 수 있다.
* 환경을 만든 후 reset()메서드로 꼭 초기화 해야한다.
* cartpole환경의 각 관측은 네 개의 실수를 담은 1D 넘파이 배열이다.
  * 이 실수는 키트의 수평 위치,카트의 속도,막대의 각도, 막대의 각속도를 나타낸다.

In [4]:
gym.envs.registry.all()# gym 모든 환경

dict_values([EnvSpec(Copy-v0), EnvSpec(RepeatCopy-v0), EnvSpec(ReversedAddition-v0), EnvSpec(ReversedAddition3-v0), EnvSpec(DuplicatedInput-v0), EnvSpec(Reverse-v0), EnvSpec(CartPole-v0), EnvSpec(CartPole-v1), EnvSpec(MountainCar-v0), EnvSpec(MountainCarContinuous-v0), EnvSpec(Pendulum-v0), EnvSpec(Acrobot-v1), EnvSpec(LunarLander-v2), EnvSpec(LunarLanderContinuous-v2), EnvSpec(BipedalWalker-v3), EnvSpec(BipedalWalkerHardcore-v3), EnvSpec(CarRacing-v0), EnvSpec(Blackjack-v0), EnvSpec(KellyCoinflip-v0), EnvSpec(KellyCoinflipGeneralized-v0), EnvSpec(FrozenLake-v0), EnvSpec(FrozenLake8x8-v0), EnvSpec(CliffWalking-v0), EnvSpec(NChain-v0), EnvSpec(Roulette-v0), EnvSpec(Taxi-v3), EnvSpec(GuessingGame-v0), EnvSpec(HotterColder-v0), EnvSpec(Reacher-v2), EnvSpec(Pusher-v2), EnvSpec(Thrower-v2), EnvSpec(Striker-v2), EnvSpec(InvertedPendulum-v2), EnvSpec(InvertedDoublePendulum-v2), EnvSpec(HalfCheetah-v2), EnvSpec(HalfCheetah-v3), EnvSpec(Hopper-v2), EnvSpec(Hopper-v3), EnvSpec(Swimmer-v2), EnvSp

In [5]:
try:
    import pyvirtualdisplay
    display = pyvirtualdisplay.Display(visible=0, size=(1400, 900)).start()
except ImportError:
    pass

env.render()

EasyProcessError: ignored